In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import iqr, randint, uniform
import lightgbm as lgb
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, SGDClassifier
from sklearn.preprocessing import Imputer, StandardScaler, QuantileTransformer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import SelectKBest, f_classif
import pickle
from skopt import gp_minimize
from skopt.plots import plot_convergence
import gc
from IPython.display import display
import warnings

warnings.simplefilter("ignore")

pd.options.display.max_columns = None

%load_ext autotime

gc.enable()
np.random.seed(235)

path = "/Users/dsaxton/home_credit_default/"

impute = Imputer(strategy="median")
quant = QuantileTransformer(output_distribution="normal")
stand = StandardScaler()

def get_imp(clf, cols):
    frame = (pd.DataFrame({"Variable": cols, 
                        "Importance": clf.feature_importances_}).
             sort_values(by="Importance", ascending=False).
             reset_index(drop=True))
    
    return frame

#### Load the data

In [2]:
df = pd.read_csv(path + "train.csv")
sk_id_curr = df.pop("SK_ID_CURR")
y = df.pop("TARGET")

time: 56.2 s


# LightGBM

#### Select features

In [3]:
clf = lgb.LGBMClassifier(n_estimators=1000, num_leaves=23, subsample=0.5)
clf.fit(df, y)

lgb_cols = df.columns[clf.feature_importances_ > 0]
len(lgb_cols)

519

time: 3min 55s


In [4]:
var_imp = get_imp(clf, df.columns)
var_imp.head(30)

,Variable,Importance
0,EXT_SOURCE_3,325
1,EXT_SOURCE_2,320
2,EXT_SOURCE_1_DIV_DAYS_BIRTH,217
3,AMT_CREDIT_DIV_AMT_ANNUITY,210
4,AVG_AGG_SYNTHETIC_TARGET,194
5,DAYS_REGISTRATION_DIV_DAYS_LAST_PHONE_CHANGE,180
6,DAYS_ID_PUBLISH,153
7,MAX_AGG_SYNTHETIC_TARGET,143
8,EXT_SOURCE_PROD,138
9,MIN_DAYS_ENTRY_PAYMENT_DIFF_DAYS_INSTALMENT_12M,137


time: 41.6 ms


In [8]:
var_imp[var_imp["Variable"].str.contains("PROJ")]

,Variable,Importance
164,ABS_RAND_PROJ9,55
193,ABS_RAND_PROJ1,45
203,RAND_PROJ9,42
213,RAND_PROJ1,39
221,ABS_RAND_PROJ2,36
258,RAND_PROJ2,28
259,ABS_RAND_PROJ4,28
260,ABS_RAND_PROJ5,28
281,RAND_PROJ6,23
285,RAND_PROJ5,23


time: 16.3 ms
